Description:<br>
Based on the MNIST dataset, design and implement classifiers including: <br>
1) least squares with regularization<br>
2) Fisher discriminant analysis (with kernels)<br>
3) Perceptron (with kernels)<br>
4) logistic regression<br>
5) SVM (with kernels)<br>
6) MLP-NN with two different error functions 

### MLP-NN with two different error functions

Load Data

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Change the input data to be suitable for network input

In [3]:
X_train = X_train.reshape((60000, 28 * 28))
X_train = X_train.astype('float32') / 255

X_test = X_test.reshape((10000, 28 * 28))
X_test = X_test.astype('float32') / 255

Constructing the neural network, the network contains two dense layers, they are fully connected. The first layer use activation function ```relu(x) = max(x, 0)```The second (and last) layer is a 10-way softmax layer, which means it will return an array of 10 probability scores (summing to 1). Each score will be the probability that the current digit image belongs to one of our 10 digit classes.

In [4]:
from keras import models
from keras import layers

network1 = models.Sequential()
network1.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network1.add(layers.Dense(10, activation='softmax'))

Compiling network1, use loss function ```categorical_crossentropy```

In [5]:
network1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Prepare the label

In [6]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Training network1, the network will start to iterate on the training data in mini-batches of 128 samples, 5 times over (each iteration over all the training data is called an epoch). At each iteration, the network will compute the gradients of the weights with regard to the loss on the batch, and update the weights accordingly. After these 5 epochs, the network will have performed 2,345 gradient updates (469 per epoch)

In [7]:
network1.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 7s 110us/step - loss: 0.2581 - acc: 0.9259
Epoch 2/5
60000/60000 [==============================] - 6s 98us/step - loss: 0.1036 - acc: 0.9701
Epoch 3/5
60000/60000 [==============================] - 6s 97us/step - loss: 0.0684 - acc: 0.9796
Epoch 4/5
60000/60000 [==============================] - 5s 92us/step - loss: 0.0501 - acc: 0.9847
Epoch 5/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.0374 - acc: 0.9889


Construct network2 as above, but use loss function ```categorical_hinge```

In [8]:
network2 = models.Sequential()
network2.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network2.add(layers.Dense(10, activation='softmax'))
network2.compile(optimizer='rmsprop', loss='categorical_hinge', metrics=['accuracy'])
network2.fit(X_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 6s 98us/step - loss: 0.2283 - acc: 0.8973
Epoch 2/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.0986 - acc: 0.9562
Epoch 3/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.0704 - acc: 0.9686
Epoch 4/5
60000/60000 [==============================] - 5s 90us/step - loss: 0.0556 - acc: 0.9755
Epoch 5/5
60000/60000 [==============================] - 6s 93us/step - loss: 0.0461 - acc: 0.9794


Now test the networks with testset

In [9]:
test_loss1, test_acc1 = network1.evaluate(X_test, y_test)
print('test_acc of network1: ', test_acc1)
test_loss2, test_acc2 = network2.evaluate(X_test, y_test)
print('test_acc of network2: ', test_acc2)

10000/10000 [==============================] - 1s 74us/step
test_acc of network1:  0.9793
10000/10000 [==============================] - 1s 72us/step
test_acc of network2:  0.9735


### SVM (with kernels) 

Use the Gaussian kernel

In [10]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000, 28 * 28))
X_train = X_train.astype('float32') / 255

X_test = X_test.reshape((10000, 28 * 28))
X_test = X_test.astype('float32') / 255

from sklearn.svm import SVC

svc_model = SVC(C=100.0, kernel='rbf', gamma=0.03)
svc_model.fit(X_train, y_train)

y_pred_svc = [int(a) for a in svc_model.predict(X_test)]
correct = sum(int(x == y) for x, y in zip(y_pred_svc, y_test))

print("%s of %s test values correct.", (correct, len(y_test)))

%s of %s test values correct. (9857, 10000)


### Least squares with regularization (Ridge)###

In [12]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

for alpha in [0.001, 0.01, 0.1, 0.5, 1]:
    ridge_model = RidgeClassifier(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    y_pred = ridge_model.predict(X_test)
    print("The accuracy score of alpha = ", alpha, " is ", accuracy_score(y_test, y_pred))

C:\Users\28347\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.3002339205314684e-09 / 5.960464477539063e-08
  RuntimeWarning)


The accuracy score of alpha =  0.001  is  0.8602


C:\Users\28347\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.3013183419730012e-08 / 5.960464477539063e-08
  RuntimeWarning)


The accuracy score of alpha =  0.01  is  0.8603
The accuracy score of alpha =  0.1  is  0.8604
The accuracy score of alpha =  0.5  is  0.8605
The accuracy score of alpha =  1  is  0.8604


As we can see, least squares with regularization (ridge) get a relatively higher accuracy score when ```alpha = 0.5```

### Logistic regression

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
penalty_options = ['l1', 'l2']
tol_options = [0.0001, 0.00001, 0.000001, 0.0000001]
param_options = dict(penalty=penalty_options,tol=tol_options)
grid = GridSearchCV(lr_model, param_options, cv=10, scoring='accuracy', verbose=1)
X_train = X_train[:1000,:]
y_train = y_train[:1000]
grid.fit(X_train, y_train)

print('Best score is: ', str(grid.best_score_))
print('Best params are: ', str(grid.best_params_))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:   44.7s finished


Best score is:  0.862
Best params are:  {'penalty': 'l2', 'tol': 0.0001}


### Perceptron (with kernels)

The reference artical is ```Jianhua Xu and Xuegong Zhang, "A multiclass kernel perceptron algorithm," 2005 International Conference on Neural Networks and Brain, Beijing, 2005, pp. 717-721.doi: 10.1109/ICNNB.2005.1614728```. I implemented the kernel perceptron all over with python. But I realized if use kernel perceptron to handle MNIST, the model has to save whole training dataset to do prediction, and one single prediction is too costly to endure not to mention the fitting process (the magnitude is bigger than billion). So I choose to analyse the ```iris``` dataset to test my code.

Load dataset ```iris```

In [19]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

My code of kernel perceptron

In [22]:
import numpy as np

class KernelPerceptron:
    '''
    :parameter
    kernel: the kernel function, default as rbf, can choose poly and linear otherwise
    param: the param of sigma**2 (rbf) or the degree of polynomial, default as 3
    '''

    def __init__(self, kernel='rbf', param=3):
        self.c_ = 0             # the c classes problem
        self.alpha_ = np.zeros(1)
        self.beta_ = np.zeros(1)
        self.N_ = 0             # the number of training dataset
        self.k_ = np.zeros(1)   # store the training dataset to predict new input
        if kernel == 'linear':
            self.method_ = self.__linearKernel
        elif kernel == 'rbf':
            self.param_ = param
            self.method_ = self.__rbfKernel
        else:
            self.param_ = param
            self.method_ = self.__polyKernel

    def __linearKernel(self, a, b):
        return np.dot(a, b)

    def __polyKernel(self, a, b):
        return np.dot(a, b) ** self.param_

    def __rbfKernel(self, a, b):
        return np.exp(- sum((a - b) ** 2) / (2 * self.param_))

    def fit(self, X_train, y_train):    # input y_train data type is float 0,1,2...
        labels = set()
        for t in y_train:            # get the number of classes
            if t not in labels:
                labels.add(t)
        self.N_ = len(X_train)
        self.c_ = len(labels)        # The number of classes
        self.alpha_ = np.zeros((self.c_, self.N_))
        self.beta_ = np.zeros((self.c_, 1))

        gram = np.zeros((self.N_, self.N_))      # calculate the gram matrix
        for i in range(self.N_):
            for j in range(i + 1):
                gram[i][j] = self.method_(X_train[i], X_train[j])
                if i != j:
                    gram[j][i] = gram[i][j]

        for i in range(self.N_):
            x = X_train[i]          # exam sample x
            ti = y_train[i]         # target value ti
            y = np.dot(self.alpha_, gram[i][:, np.newaxis]) + self.beta_
            j = np.where(y == max(y))[0][0]             # the index of the biggest
            if j != ti:                                 # misclassified sample
                for m in range(self.N_):
                    self.alpha_[ti][m] += np.exp(- sum((X_train[m] - x) ** 2) / 2)
                    self.alpha_[j][m] -= np.exp(- sum((X_train[m] - x) ** 2) / 2)
                self.beta_[ti] += 1
                self.beta_[j] -= 1

        # The training has been done, now store X_train for the future prediction
        self.k_ = X_train

    def __y(self, x):
        kx = np.zeros(self.N_)
        for i in range(self.N_):
            kx[i] = self.method_(self.k_[i], x)
        kx = kx[:, np.newaxis]
        y = np.dot(self.alpha_, kx) + self.beta_
        return np.where(y == max(y))[0][0]

    def predict(self, X_test):
        total = len(X_test)
        y_pred = np.zeros(total)
        for i in range(total):
            y_pred[i] = self.__y(X_test[i])
        return y_pred

Test my code

In [25]:
kp_model = KernelPerceptron(kernel='rbf', param=1)
kp_model.fit(X_train, y_train)
y_pred_kp = kp_model.predict(X_test)

from sklearn.metrics import accuracy_score
print('The accuracy score of my kernel perceptron is ', accuracy_score(y_test, y_pred_kp))

The accuracy score of my kernel perceptron is  0.9736842105263158


### Fisher discriminant analysis (with kernels) ###

I inplemented the algorithm mentioned in ```https://en.wikipedia.org/wiki/Kernel_Fisher_discriminant_analysis#cite_note-texture-7```The kernel fisher solution confronts the same problem as kernel perceptron does. So I will test my code using ```iris```dataset instead.

My code of kernel fisher

In [26]:
from scipy.linalg import eigh as largest_eigh

class KernelFisher:
    def __init__(self, kernel='rbf', param=3):
        self.yMeans_ = []           # yMeans[i] is the projected mean for class i
        self.X_ = []                # store the training dataset for prediction
        self.AstarT_ = np.mat(1)    # The A*T used in ultimate decision function
        if kernel == 'linear':
            self.method_ = self.__linearKernel
        elif kernel == 'rbf':
            self.param_ = param
            self.method_ = self.__rbfKernel
        else:
            self.param_ = param
            self.method_ = self.__polyKernel

    def __linearKernel(self, a, b):
        return np.dot(a, b)

    def __polyKernel(self, a, b):
        return np.dot(a, b) ** self.param_

    def __rbfKernel(self, a, b):
        return np.exp(- sum((a - b) ** 2) / (2 * self.param_))

    def __y(self, xt):
        yt = []
        for xi in self.X_:
            yt.append(self.method_(xi, xt))
        return np.array(self.AstarT_ * np.mat(yt).T)

    def fit(self, X_train, y_train):
        self.X_ = X_train
        n = len(X_train)                  # size of training dataset
        c = int(max(y_train)) + 1         # y_train must be coded as 0,1,2...
        Ms = []                           # Ms[i] = Mi
        Ks = []                           # Ks[j] = Kj
        sortedByClass = []         # sortedByClass[i] is a np.array contains all points belong to class i

        for y in range(c):
            currentClass = []
            for i in range(n):
                if y_train[i] == y:
                    currentClass.append(X_train[i])
            sortedByClass.append(np.array(currentClass))

        for j in range(c):
            kj = []
            for xi in X_train:
                rowi = []
                for xj in sortedByClass[j]:
                    rowi.append(self.method_(xi, xj))
                kj.append(np.array(rowi))
            Ks.append(np.array(kj))

        for i in range(c):
            Mi = []
            for xi in X_train:
                sum = 0
                for xj in sortedByClass[i]:
                    sum += self.method_(xi, xj)
                Mi.append(sum)
            Ms.append(np.array(Mi)[:, np.newaxis] / len(sortedByClass[i]))

        Mstar = []

        for xi in X_train:
            sum = 0
            for xj in X_train:
                sum += self.method_(xi, xj)
            Mstar.append(sum)

        Mstar = np.array(Mstar)[:, np.newaxis] / n
        M = np.zeros((n, n))

        for i in range(c):
            M += (Ms[i] - Mstar) * (Ms[i] - Mstar).T / len(sortedByClass[i])

        reversedN = np.zeros((n, n))            # the reversed matrix N

        for j in range(c):
            Nj = len(sortedByClass[j])
            reversedN += np.dot(np.dot(Ks[j], (np.eye(Nj) - np.full((Nj, Nj), 1 / Nj))), Ks[j].T)

        reversedN = np.mat(reversedN).I

        evals, self.AstarT_ = largest_eigh(reversedN * M, eigvals=(n - c + 1, n - 1))
        self.AstarT_ = self.AstarT_.T

        for j in range(c):
            m = 0
            for xi in sortedByClass[j]:
                m += self.__y(xi)
            m /= len(sortedByClass[j])
            self.yMeans_.append(np.array(m))

    def predict(self, X_test):
        y_pred = []
        for xt in X_test:
            dis = []
            yt = self.__y(xt)
            for mj in self.yMeans_:
                dis.append(np.sum(np.abs(yt - mj)))
            y_pred.append(dis.index(min(dis)))
        return np.array(y_pred)

Test my code 

In [27]:
kf_model = KernelFisher(kernel='rbf', param=1)
kf_model.fit(X_train, y_train)
y_pred_kf = kf_model.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred_kf))

0.8947368421052632
